In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
import shutil
from pathlib import Path
from sort_alpha import sorted_alpha
import random
from scipy.special import kl_div
from scipy.spatial import distance
from sklearn.cluster import KMeans

In [ ]:
def torso_size(data):
    neck = [data['x'].loc[1], data['y'].loc[1]]
    if neck ==0:
        return([0])
    hip1 = [data['x'].loc[8], data['y'].loc[8]]
    hip2 = [data['x'].loc[11], data['y'].loc[11]]
    if hip1[0]>0 and hip2[0]>0:
        probabilities = [data['p'].loc[8], data['p'].loc[11]]
        max_p = np.argmax(probabilities)
        if max_p==0:
            torso = distance.euclidean(hip1, neck)
        else:
            torso = distance.euclidean(hip2, neck)
    elif hip1[0]==0:
        torso = distance.euclidean(hip2, neck)
    else:
        torso = distance.euclidean(hip1, neck)
    return(torso)

In [ ]:
def CountQuads(Arr2D, point):
    """ Computes the probabilities of finding points in each 4 quadrant
    defined by a vertical and horizontal lines crossing the point, by counting
    the proportion of points in Arr2D in each quadrant.
    :param list Arr2D: Array of points to be counted.
    :param array point: A 2 element list, point, which is the center of
    4 square quadrants.
    :returns: a tuple of 4 floats.  The probabilities of finding a point in
    each quadrants, with point as the origin.  p stands for positive, n for
    negative, with the first and second positions meaning the x and y
    directions respectively.
    """
    if isinstance(point, list):
        point = np.asarray((np.ravel(point)))
    elif type(point).__module__+type(point).__name__ == 'numpyndarray':
        point = np.ravel(point.copy())
    else:
        raise TypeError('Input point is neither list nor numpyndarray')
    if len(point) != 2:
        return
    if isinstance(Arr2D, list):
        Arr2D = np.asarray((Arr2D))
    elif type(Arr2D).__module__+type(Arr2D).__name__ == 'numpyndarray':
        pass
    else:
        raise TypeError('Input Arr2D is neither list nor numpyndarray')
    if Arr2D.shape[1] > Arr2D.shape[0]:  # Reshape to A[row,column]
        Arr2D = Arr2D.copy().T
    if Arr2D.shape[1] != 2:
        raise TypeError('Input Arr2D is not 2D')
    # The pp of Qpp refer to p for 'positive' and n for 'negative' quadrants.
    # In order. first subscript is x, second is y.
    Qpp = Arr2D[(Arr2D[:, 0] > point[0]) & (Arr2D[:, 1] > point[1]), :]
    Qnp = Arr2D[(Arr2D[:, 0] < point[0]) & (Arr2D[:, 1] > point[1]), :]
    Qpn = Arr2D[(Arr2D[:, 0] > point[0]) & (Arr2D[:, 1] < point[1]), :]
    Qnn = Arr2D[(Arr2D[:, 0] < point[0]) & (Arr2D[:, 1] < point[1]), :]
    # Normalized fractions:
    ff = 1./len(Arr2D)
    fpp = len(Qpp)*ff
    fnp = len(Qnp)*ff
    fpn = len(Qpn)*ff
    fnn = len(Qnn)*ff
    # NOTE:  all the f's are supposed to sum to 1.0. Float representation
    # cause SOMETIMES sum to 1.000000002 or something. I don't know how to
    # test for that reliably, OR what to do about it yet. Keep in mind.
    return(fpp, fnp, fpn, fnn)


def FuncQuads(func2D, point, xlim, ylim, rounddig=4):
    """ Computes the probabilities of finding points in each 4 quadrant
    defined by a vertical and horizontal lines crossing the point, by
    integrating the density function func2D in each quadrant.
    :param array func2D: Density function that takes 2 arguments: x and y.
    :param list point: A 2 element list, point, which is the center of 4
    square quadrants.
    :param array xlim,ylim: Domain of numerical integration necessary to
    compute the quadrant probabilities.
    :returns: a tuple of 4 floats. The probabilities of finding a point in
    each quadrants, with point as the origin.  p stands for positive,
    n for negative, with the first and second positions meaning the x and y
    directions respectively.
    """
    if callable(func2D):
        if len(inspect.getfullargspec(func2D)[0]) != 2:
            raise TypeError('Input func2D is not a function with 2 arguments')
        pass
    else:
        raise TypeError('Input func2D is not a function')
    # If xlim, ylim and point are not lists or ndarray, exit.
    if isinstance(point, list):
        point = np.asarray((np.ravel(point)))
    elif type(point).__module__+type(point).__name__ == 'numpyndarray':
        point = np.ravel(point.copy())
    else:
        raise TypeError('Input point is not a list or numpyndarray')
    if len(point) != 2:
        raise TypeError('Input point has not exactly 2 elements')
    if isinstance(xlim, list):
        xlim = np.asarray((np.sort(np.ravel(xlim))))
    elif type(xlim).__module__+type(xlim).__name__ == 'numpyndarray':
        xlim = np.sort(np.ravel(xlim.copy()))
    else:
        raise TypeError('Input xlim is not a list or ndarray')
    if len(xlim) != 2:
        raise TypeError('Input xlim has not exactly 2 elements')
    if xlim[0] == xlim[1]:
        raise TypeError('Input xlim[0] should be different to xlim[1]')
    if isinstance(ylim, list):
        ylim = np.asarray((np.sort(np.ravel(ylim))))
    elif type(ylim).__module__+type(ylim).__name__ == 'numpyndarray':
        ylim = np.sort(np.ravel(ylim.copy()))
    else:
        raise TypeError('Input ylim is not a list or ndarray')
    if len(ylim) != 2:
        raise TypeError('Input ylim has not exactly 2 elements')
    if ylim[0] == ylim[1]:
        raise TypeError('Input ylim[0] should be different to ylim[1]')
    # Numerical integration to find the quadrant probabilities.
    totInt = scipy.integrate.dblquad(func2D, *xlim,
                                     lambda x: np.amin(ylim),
                                     lambda x: np.amax(ylim))[0]
    Qpp = scipy.integrate.dblquad(func2D, point[0], np.amax(xlim),
                                  lambda x: point[1],
                                  lambda x: np.amax(ylim))[0]
    Qpn = scipy.integrate.dblquad(func2D, point[0], np.amax(xlim),
                                  lambda x: np.amin(ylim),
                                  lambda x: point[1])[0]
    Qnp = scipy.integrate.dblquad(func2D, np.amin(xlim), point[0],
                                  lambda x: point[1],
                                  lambda x: np.amax(ylim))[0]
    Qnn = scipy.integrate.dblquad(func2D, np.amin(xlim), point[0],
                                  lambda x: np.amin(ylim),
                                  lambda x: point[1])[0]
    fpp = round(Qpp/totInt, rounddig)
    fnp = round(Qnp/totInt, rounddig)
    fpn = round(Qpn/totInt, rounddig)
    fnn = round(Qnn/totInt, rounddig)
    return(fpp, fnp, fpn, fnn)


def Qks(alam, iter=100, prec=1e-17):
    """ Computes the value of the KS probability function, as a function of
    alam, the D statistic. From *Numerical recipes in C* page 623: '[...]
    the K–S statistic useful is that its distribution in the case of the null
    hypothesis (data sets drawn from the same distribution) can be calculated,
    at least to useful approximation, thus giving the significance of any
    observed nonzero value of D.' (D being the KS statistic).
    :param float alam: D statistic.
    :param int iter: Number of iterations to be perfomed. On non-convergence,
    returns 1.0.
    :param float prec: Convergence criteria of the qks. Stops converging if
    that precision is attained.
    :returns: a float. The significance level of the observed D statistic.
    """
    # If j iterations are performed, meaning that toadd
    # is still 2 times larger than the precision.
    if isinstance(alam, int) | isinstance(alam, float):
        pass
    else:
        raise TypeError('Input alam is neither int nor float')
    toadd = [1]
    qks = 0.
    j = 1
    while (j < iter) & (abs(toadd[-1]) > prec*2):
        toadd.append(2.*(-1.)**(j-1.)*np.exp(-2.*j**2.*alam**2.))
        qks += toadd[-1]
        j += 1
    if (j == iter) | (qks > 1):  # If no convergence after j iter, return 1.0
        return(1.0)
    if qks < prec:
        return(0.)
    else:
        return(qks)







def ks2d2s(Arr2D1, Arr2D2):
    """ ks stands for Kolmogorov-Smirnov, 2d for 2 dimensional,
    2s for 2 samples.
    KS test for goodness-of-fit on two 2D samples. Tests the hypothesis that
    the two samples are from the same distribution.
    :param array Arr2D1: 2D array of points/samples.
    :param array Arr2D2: 2D array of points/samples.
    :returns: a tuple of two floats. First, the two-sample K-S statistic.
    If this value is higher than the significance level of the hypothesis,
    it is rejected. Second, the significance level of *d*. Small values of
    prob show that the two samples are significantly different.
    """
    if type(Arr2D1).__module__+type(Arr2D1).__name__ == 'numpyndarray':
        pass
    else:
        raise TypeError('Input Arr2D1 is neither list nor numpyndarray')
    if Arr2D1.shape[1] > Arr2D1.shape[0]:
        Arr2D1 = Arr2D1.copy().T
    if type(Arr2D2).__module__+type(Arr2D2).__name__ == 'numpyndarray':
        pass
    else:
        raise TypeError('Input Arr2D2 is neither list nor numpyndarray')
    if Arr2D2.shape[1] > Arr2D2.shape[0]:
        Arr2D2 = Arr2D2.copy().T
    if Arr2D1.shape[1] != 2:
        raise TypeError('Input Arr2D1 is not 2D')
    if Arr2D2.shape[1] != 2:
        raise TypeError('Input Arr2D2 is not 2D')
    d1, d2 = 0., 0.
    for point1 in Arr2D1:
        fpp1, fmp1, fpm1, fmm1 = CountQuads(Arr2D1, point1)
        fpp2, fmp2, fpm2, fmm2 = CountQuads(Arr2D2, point1)
        d1 = max(d1, abs(fpp1-fpp2))
        d1 = max(d1, abs(fpm1-fpm2))
        d1 = max(d1, abs(fmp1-fmp2))
        d1 = max(d1, abs(fmm1-fmm2))
    for point2 in Arr2D2:
        fpp1, fmp1, fpm1, fmm1 = CountQuads(Arr2D1, point2)
        fpp2, fmp2, fpm2, fmm2 = CountQuads(Arr2D2, point2)
        d2 = max(d2, abs(fpp1-fpp2))
        d2 = max(d2, abs(fpm1-fpm2))
        d2 = max(d2, abs(fmp1-fmp2))
        d2 = max(d2, abs(fmm1-fmm2))
    d = (d1+d2)/2.
    sqen = np.sqrt(len(Arr2D1)*len(Arr2D2)/(len(Arr2D1)+len(Arr2D2)))
    R1 = scipy.stats.pearsonr(Arr2D1[:, 0], Arr2D1[:, 1])[0]
    R2 = scipy.stats.pearsonr(Arr2D2[:, 0], Arr2D2[:, 1])[0]
    RR = np.sqrt(1.-(R1*R1+R2*R2)/2.)
    prob = Qks(d*sqen/(1.+RR*(0.25-0.75/sqen)))
    # Small values of prob show that the two samples are significantly
    # different. Prob is the significance level of an observed value of d.
    # NOT the same as the significance level that ou set and compare to D.
    return(d, prob)

In [ ]:
path_openpose = Path('/Users/andreibirladeanu/Documents/Data/bounding_boxes/1043_meal')
folders = sorted_alpha(path_openpose)

In [ ]:
means = []
for folder in folders:
    temp_list= []
    for coord in sorted_alpha(os.path.join(path_openpose, folder)):
        image = cv2.imread(os.path.join(path_openpose, folder, coord))
        mean = np.mean(image,axis=2, dtype=np.float32)
        temp_list.append(np.mean(mean))
    means.append(temp_list)

In [ ]:
import sys
import inspect
import numpy as np
import scipy.stats

ks2d2s(np.array(locations[0]), np.array(locations[1]))

In [ ]:
means[0]

In [ ]:
np.mean(kl_div(means[0], means[1]))

In [ ]:
def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)


In [ ]:
from norfair import Detection, Tracker, Video, draw_tracked_objects, draw_tracked_boxes
from norfair import FilterPyKalmanFilterFactory
from norfair import OptimizedKalmanFilterFactory
from norfair import NoFilterFactory

In [ ]:
video = Video(input_path="/Users/andreibirladeanu/Documents/Data/meal_videos/1053_meal.mp4")
tracker = Tracker(distance_function=euclidean_distance, distance_threshold=100, hit_counter_max=100, initialization_delay =0, 
                 past_detections_length=10)

In [ ]:
path_openpose = '/Users/andreibirladeanu/Documents/Data/meal_tracked_bbox/1053_meal/skel1'
coords = sorted_alpha(path_openpose)

In [ ]:
filter = NoFilterFactory(create_filter(detections))

In [ ]:
points = []
scores=[]
for coord in coords:
    data = pd.read_csv(os.path.join(path_openpose,coord))
    points.append(data[['x','y']].to_numpy())
    scores.append(data['p'].to_numpy())

In [ ]:
coords_frames = [int(x.split(".")[0]) for x in coords]

In [ ]:
cap = cv2.VideoCapture("/Users/andreibirladeanu/Documents/Data/meal_videos/1053_meal.mp4")
tracked_ = []
for point,score,coord in zip(points,scores,coords_frames):
    detections = [Detection(points=point,scores=score)]
    tracked_objects = tracker.update(detections=detections)
    tracked_.append(tracked_objects)
    print(coord, tracked_objects)
    cap.set(1,coord-1)
    ret,image = cap.read()
    draw_tracked_boxes(image, tracked_objects, id_size =2, draw_box=True, draw_labels=True)
    cv2.imshow('frame'+str(coord),image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    video.write(image)

In [ ]:
### do k means on the lenghts 
path_openpose = '/Users/andreibirladeanu/Documents/Data/tracking/skeletons/meal_final_tracking/'
folders = sorted_alpha(path_openpose)


In [ ]:
coords_1 = []
for coord in sorted_alpha(os.path.join(path_openpose,folders[0])):
    data = pd.read_csv(os.path.join(path_openpose,folders[0],coord))
    neck = [data['x'].loc[1], data['y'].loc[1]]
    coords_1.append(np.array(neck))

In [ ]:
coords_2 = []
for coord in sorted_alpha(os.path.join(path_openpose,folders[1])):
    data = pd.read_csv(os.path.join(path_openpose,folders[1],coord))
    neck = [data['x'].loc[1], data['y'].loc[1]]
    coords_2.append(np.array(neck))

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
labels_1 = [0 for i in range(len(coords_1))]
labels_2 = [1 for i in range(len(coords_2))]

In [ ]:
X = np.concatenate((coords_1, coords_2))

In [ ]:
len(labels_2), len(coords_2)

In [ ]:
len(np.concatenate((labels_1, labels_2)))

In [ ]:
samples = silhouette_samples(X, np.concatenate((labels_1, labels_2)))

In [ ]:
silhouette_score(X, np.concatenate((labels_1, labels_2)))

In [ ]:
samples

In [ ]:
### do it for the whole sampel now 
similarity = []
for folder in tqdm(folders): 
    if len(sorted_alpha(os.path.join(path_openpose,folder)))<2:
        continue
    skel1 = []
    skel2 = []
    for skel in enumerate(sorted_alpha(os.path.join(path_openpose,folder))):
        for coord in sorted_alpha(os.path.join(path_openpose,folder,skel[1])):
            data = pd.read_csv(os.path.join(path_openpose,folder,skel[1], coord))
            neck = [data['x'].loc[1], data['y'].loc[1]]
            if skel[0]== 0:
                skel1.append(neck)
            else:
                skel2.append(neck)
    labels_1 = [0 for i in range(len(skel1))]
    labels_2 = [1 for i in range(len(skel2))]
    labels = np.concatenate((labels_1, labels_2))
    X = np.concatenate((skel1, skel2))
    score = silhouette_score(X, labels)
    similarity.append(score)
            

In [ ]:
### how many humans tracked
path_openpose = '/Users/andreibirladeanu/Documents/Data/tracking/skeletons/meal_final_tracking/'
folders = sorted_alpha(path_openpose)

In [ ]:
folders

In [ ]:
skel1 = []
skel2 = []
for folder in folders:
    if len(sorted_alpha(os.path.join(path_openpose,folder)))<2:
        continue
    skeletons=[]
    for skeleton in enumerate(sorted_alpha(os.path.join(path_openpose,folder))):
        if skeleton[0]==0:
            skel1.append(len(os.listdir(os.path.join(path_openpose,folder,skeleton[1]))))
        else:
            skel2.append(len(os.listdir(os.path.join(path_openpose,folder,skeleton[1]))))

In [ ]:
len(skel1), len(skel2)

In [ ]:
folders.remove('1091_meal')

In [ ]:
np.mean(similarity)

In [ ]:
humans = pd.DataFrame(list(zip(folders,skel1,skel2, similarity)), columns=['video', 'human1','human2','similarity'])

In [ ]:
humans

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(humans)

In [ ]:
humans.to_csv('tracking_stats.csv')

In [ ]:
### another clustering based on rgb channels
path_folders = '/Users/andreibirladeanu/Documents/Data/tracking/bboxes/final_bbox_clustering'
path_videos = '/Users/andreibirladeanu/Documents/Data/meal_videos'
similarity = []
for folder in folders:
    print(folder)
    skel1 = []
    skel2 = []
    if len(sorted_alpha(os.path.join(path_openpose,folder)))<2:
        continue
    for skel in enumerate(sorted_alpha(os.path.join(path_folders,folder))):
        for coord in sorted_alpha(os.path.join(path_folders,folder,skel[1])):
            img = cv2.imread(os.path.join(path_folders, folder, skel[1], coord))
            b,g,r = cv2.split(img)
            if skel[0]== 0:
                skel1.append([np.mean(b), np.mean(g), np.mean(r)])
            else:
                skel2.append([np.mean(b), np.mean(g), np.mean(r)])
    labels_1 = [0 for i in range(len(skel1))]
    labels_2 = [1 for i in range(len(skel2))]
    labels = np.concatenate((labels_1, labels_2))
    X = np.concatenate((skel1, skel2))
    score = silhouette_score(X, labels)
    print(score)
    similarity.append(score)
            
            

In [ ]:

for x , y in zip(np.round_(similarity, decimals=2), folders):
    if x <=0.15:
        print(x,y)

In [ ]:
len(folders)

In [ ]:
round(np.array(similarity))

In [ ]:
huh =np.round_(similarity, decimals=2)

In [ ]:
huh[0]==0.63

In [ ]:
len(folders)